In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [7]:
X_train = pd.read_csv("X_train.csv", sep = ',')
y_train = pd.read_csv("y_train.csv", sep = ',')
X_test = pd.read_csv("X_test.csv", sep = ',')
y_test = pd.read_csv("y_test.csv", sep = ',')

merged_train_data = pd.concat([X_train, y_train], axis=1)
train_data = merged_train_data

merged_test_data = pd.concat([X_test, y_test], axis=1)
test_data = merged_test_data

In [8]:
oneR_train_data = train_data.copy()

In [9]:
def get_predictor_value(dataframe, column):
    split = (dataframe[column][dataframe['ack'] == 0].mean() + dataframe[column][dataframe['ack'] == 1].mean())/2
    group = dataframe[dataframe[column] > split].groupby('ack').size()
    if group[0] > group[1]:
        left = 1
        right = 0
    else:
        left = 0
        right = 1

    return split, left, right

def oneR(train_data, column, data):
    train_data.loc[train_data[column] < data[0], 'temp'] = data[1]
    train_data.loc[train_data[column] >= data[0], 'temp'] = data[2]
    accuracy = accuracy_score(train_data['ack'], train_data['temp'])
    precision = precision_score(train_data['ack'], train_data['temp'])
    recall = recall_score(train_data['ack'], train_data['temp'])
    total=(accuracy + precision + recall) / 3
    train_data.drop('temp', axis=1, inplace=True)
    return [accuracy, precision, recall,total]

In [10]:
best_accuracy = 0 
for column in oneR_train_data.columns:
    if column == 'ack':
        continue
    data = get_predictor_value(oneR_train_data, column)
    accuracy_values = oneR(oneR_train_data, column, data)
    if accuracy_values[3] > best_accuracy:
        best_column = column
        best_accuracy = accuracy_values[3]
        best_column_metrics = accuracy_values

print('best column from oneR: ' + best_column)
print('total: ' + str(best_column_metrics[3]))
print('accuracy: ' + str(best_column_metrics[0]))
print('precision: ' + str(best_column_metrics[1]))
print('recall: ' + str(best_column_metrics[2]))

best column from oneR: page_activity_duration
total: 0.900567200058147
accuracy: 0.8994723687864482
precision: 0.8568387440127727
recall: 0.9453904873752202
